Создать нейронную сеть с нуля, т.е. не используя готовые библиотеки. Пример работы на любом табличном датасете. 
Сделать класс, в котором реализована возможность задать количество нейронов в скрытом слое и провести обучение.

In [348]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Загружаем датасет с данными о заболеваниях сердца, делим на обучающую и тестовую выборку, в качестве целевой переменной беру Disease (0 - нет патологии, 1 - есть). Так же выполнила one-hot encoding и нормализацию

In [349]:
def load_dataset():
    df = pd.read_csv('heart_disease.csv')
    df_encoded = pd.get_dummies(df, drop_first=True)

    X = df_encoded.drop('Disease', axis=1).values
    y = df_encoded['Disease'].values.reshape(-1, 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    print(f"\nРазмер обучающей выборки: {X_train.shape}")
    print(f"Размер тестовой выборки: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test, df.columns.tolist()

In [350]:
# y = X·W + b
def linear_regression(X: np.ndarray, weights: np.ndarray, bias: float) -> np.ndarray:
    return np.dot(X, weights) + bias

# f(x) = 1 / (1 + exp(-x))
def activation_function(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))

# производная
def activation_derivative(x: np.ndarray) -> np.ndarray:
    fx = activation_function(x)
    return fx * (1 - fx)


Определение нейросети

In [351]:
class SimpleNeuron:
    def __init__(self, input_size: int, hidden_neurons: int = 0):
        self.input_size = input_size
        self.hidden_neurons = hidden_neurons
        scale = np.sqrt(2.0 / input_size)
        
        if hidden_neurons > 0:
            self.W1 = np.random.randn(input_size, hidden_neurons) * scale
            self.b1 = np.zeros((1, hidden_neurons))
            self.W2 = np.random.randn(hidden_neurons, 1) * np.sqrt(2.0 / hidden_neurons)
            self.b2 = 0.0
        else:
            self.weights = np.random.randn(input_size) * scale
            self.bias = 0.0
        
    def forward(self, X: np.ndarray) -> np.ndarray:
        self.X = X
        
        if self.hidden_neurons > 0:
            self.z1 = np.dot(X, self.W1) + self.b1
            self.a1 = activation_function(self.z1)
            
            self.z2 = np.dot(self.a1, self.W2) + self.b2
            self.output = activation_function(self.z2)
        else:
            self.linear_output = np.dot(X, self.weights) + self.bias
            self.output = activation_function(self.linear_output)
        
        if len(self.output.shape) == 1:
            self.output = self.output.reshape(-1, 1)
            
        return self.output
    
    def backward(self, y_true: np.ndarray, learning_rate: float = 0.01) -> float:
        if len(y_true.shape) == 1:
            y_true = y_true.reshape(-1, 1)

        m = self.X.shape[0]

        error = y_true - self.output
        
        if self.hidden_neurons > 0:
            delta2 = error * activation_derivative(self.z2)
            dW2 = np.dot(self.a1.T, delta2) / m
            db2 = np.mean(delta2, axis=0)

            delta1 = np.dot(delta2, self.W2.T) * activation_derivative(self.z1)
            dW1 = np.dot(self.X.T, delta1) / m
            db1 = np.mean(delta1, axis=0)

            self.W2 += learning_rate * dW2
            self.b2 += learning_rate * db2
            self.W1 += learning_rate * dW1
            self.b1 += learning_rate * db1.reshape(1, -1)
        else:

            delta = error * activation_derivative(self.linear_output)

            dW = np.zeros_like(self.weights)
            for i in range(m):
                dW += self.X[i] * delta[i, 0]
            dW /= m
            
            db = np.mean(delta)

            self.weights += learning_rate * dW
            self.bias += learning_rate * db

        return np.mean(error ** 2)
    
    def train(self, X: np.ndarray, y: np.ndarray, epochs: int = 1000, learning_rate: float = 0.01) -> list:
        loss_history = []
        
        for epoch in range(epochs):
            self.forward(X)
            loss = self.backward(y, learning_rate)
            loss_history.append(loss)
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, loss: {loss:.6f}")
                
        return loss_history
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return self.forward(X)

In [352]:
def test_simple_neuron(X_train, X_test, y_train, y_test, hidden_neurons=0):

    print(f"{hidden_neurons} нейронов в скрытом слое\n")

    input_size = X_train.shape[1]
    neuron = SimpleNeuron(input_size=input_size, hidden_neurons=hidden_neurons)

    loss_history = neuron.train(X_train, y_train, epochs=1000, learning_rate=0.01)

    predictions = neuron.predict(X_test)
    predicted_classes = (predictions > 0.5).astype(int)

    accuracy = np.mean(predicted_classes == y_test)

    tp = np.sum((predicted_classes == 1) & (y_test == 1))
    tn = np.sum((predicted_classes == 0) & (y_test == 0))
    fp = np.sum((predicted_classes == 1) & (y_test == 0))
    fn = np.sum((predicted_classes == 0) & (y_test == 1))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}\n")
    
    for i in range(10):
        print(f"Real {y_test[i][0]}")
        print(f"Predictions {predictions[i][0]:.4f}")
        print(f"Predicted {predicted_classes[i][0]}\n")
    
    return accuracy, precision, recall, f1

In [353]:
X_train, X_test, y_train, y_test, column_names = load_dataset()
simple_neuron_metrics = test_simple_neuron(X_train, X_test, y_train, y_test, 50)


Размер обучающей выборки: (216, 13)
Размер тестовой выборки: (54, 13)
50 нейронов в скрытом слое

Epoch 0, loss: 0.240560
Epoch 100, loss: 0.227126
Epoch 200, loss: 0.215780
Epoch 300, loss: 0.205857
Epoch 400, loss: 0.197102
Epoch 500, loss: 0.189362
Epoch 600, loss: 0.182514
Epoch 700, loss: 0.176447
Epoch 800, loss: 0.171061
Epoch 900, loss: 0.166270

Accuracy: 0.8519
Precision: 1.0000
Recall: 0.6190
F1-score: 0.7647

Real 1
Predictions 0.5715
Predicted 1

Real 1
Predictions 0.4949
Predicted 0

Real 0
Predictions 0.2321
Predicted 0

Real 0
Predictions 0.3835
Predicted 0

Real 0
Predictions 0.4380
Predicted 0

Real 1
Predictions 0.4409
Predicted 0

Real 0
Predictions 0.4385
Predicted 0

Real 0
Predictions 0.2338
Predicted 0

Real 0
Predictions 0.3932
Predicted 0

Real 0
Predictions 0.3243
Predicted 0

